In [1]:
API_KEY = 'eb2b20c374f0fd2166fb9429253388b77a9de394c70511ff3a18d46c0d850b3a'
#instantiate query api
from sec_api import QueryApi
queryApi = QueryApi(api_key=API_KEY) 

In [2]:
import sys,os
print(sys.executable)
print(os.getcwd()) 

D:\coding\Nanik\SEC\backend\virt\scripts\python.exe


NameError: name 'os' is not defined

In [ ]:
import pandas as pd

In [ ]:
def get_filings(query):
  from_param = 0
  size_param = 200
  all_filings = []

  while True:
    query['from'] = from_param
    query['size'] = size_param

    response = queryApi.get_filings(query)
    filings = response['filings']

    if len(filings) == 0:
      break

    all_filings.extend(filings)

    from_param += size_param

  return pd.json_normalize(all_filings)

In [ ]:
form_type_query  = 'formType:("10-K") AND NOT formType:("10-K/A", "10-Q/A", NT)'
ticker_query     = 'ticker:(META)' 
date_range_query = 'filedAt:[2010-01-01 TO 2025-03-15]'

lucene_query = f"{form_type_query} AND {ticker_query} AND {date_range_query}"

search_query = {
  "query": lucene_query,
  "from": "0",
  "size": "500",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

filings = get_filings(search_query)
len(filings)


In [ ]:
lucene_query

In [ ]:
search_query

In [ ]:
filings.head(20)

In [ ]:
urls = filings[['ticker',
                'formType',
                'periodOfReport',
                'filedAt',
                'linkToFilingDetails']].rename(columns={'linkToFilingDetails': 'filingUrl'})

urls['financialReportsUrl'] = urls['filingUrl'].apply(lambda url: '/'.join(url.split('/')[:-1]) + '/Financial_Report.xlsx')

urls.head(10)

In [ ]:
#Download Financial_reports.xlsx of All EDGAR Filings
import os, requests

folder_path = './reports'
API_KEY = 'eb2b20c374f0fd2166fb9429253388b77a9de394c70511ff3a18d46c0d850b3a'
if not os.path.exists(folder_path):
  os.makedirs(folder_path)


def download_report(filing,api_key):
    try:
      reports_path = filing['financialReportsUrl'].replace('https://www.sec.gov/Archives/edgar/data/', '')
      base_url = f'https://archive.sec-api.io/{reports_path}'
      render_api_url = f'{base_url}?token={api_key}'

      response = requests.get(render_api_url)
      response.raise_for_status()
      print(f"Requesting: {render_api_url}")
      print(f"Response Code: {response.status_code}, Response Text: {response.text}")


      file_name = f"{filing['ticker']}-{filing['periodOfReport']}-{filing['formType']}.xlsx"
      file_path = f"{folder_path}/{file_name}"

      with open(file_path, 'wb') as output:
              output.write(response.content)
    except Exception as e:
          print(f"Error downloading report for {filing['ticker']}: {e}")
          

In [ ]:
#!pip install -q pandarallel

from pandarallel import pandarallel
number_of_workers = 4
pandarallel.initialize(progress_bar=False, nb_workers=number_of_workers, verbose=0)

In [ ]:

urls.apply(lambda filing: download_report(filing, API_KEY), axis=1)
print(f"Downloaded {len(urls)} reports")

In [ ]:
with open("sec_finance_data_excel.ipynb", "r", encoding="utf-8", errors="replace") as f:
    content = f.read()

with open("sec_finance_data_excel_fixed.ipynb", "w", encoding="utf-8") as f:
    f.write(content)

print("Fixed notebook saved as sec_finance_data_excel_fixed.ipynb")
